# Portfolio Optimization Project

# 📑 Table of Contents

Welcome to the **Portfolio Optimization Project** — a full walk-through of modern portfolio management concepts using Python, Jupyter, and financial theory. 

---

## Part I — Foundations: Statistics & Efficient Frontiers
- [1. Statistical Analysis & KPIs](#1-statistical-analysis--kpis)  
  Compute returns, volatility, Sharpe/Sortino ratios, skewness, kurtosis, correlations, and cumulative charts.  
- [2. Two-Asset Efficient Frontier](#2-two-asset-efficient-frontier)  
  Visualize diversification benefits with a simple two-asset trade-off.  
- [3. Three-Asset Efficient Frontier](#3-three-asset-efficient-frontier)  
  Extend frontier analysis to three assets using 3D plots.  
- [4. Multi-Asset Efficient Frontier](#4-multi-asset-efficient-frontier)  
  Construct the full frontier with all assets; compare unconstrained vs. constrained versions.  

---

## Part II — Optimization & Allocation
- [5. Mean–Variance Optimization (MVO)](#5-meanvariance-optimization-mvo)  
  Implement baseline and constrained MVO using historical mean & covariance.  
- [6. Optimal Asset Allocation](#6-optimal-asset-allocation)  
  Derive efficient allocations across risk targets and compare concentration.  
- [7. Max Sharpe & Max Sortino Portfolios](#7-maximize-sharpe--sortino-portfolios)  
  Optimize explicitly for risk-adjusted return metrics and compare results.  

---

## Part III — Forward-Looking Analysis
- [8. Capital Market Expectations](#8-capital-market-expectations)  
  Introduce forward-looking assumptions and optional Black–Litterman.  
- [9. Forward-Looking Portfolio Statistics](#9-forward-looking-portfolio-statistics)  
  Recompute risk/return metrics; simulate outcomes (Monte Carlo, VaR, CVaR).  
- [10. Strategic Asset Allocation](#10-strategic-asset-allocation)  
  Define long-term policy portfolio; compare vs. optimized allocations.  

---

## Part IV — Backtesting & Evaluation
- [11. Backtesting](#11-backtesting)  
  Run rolling-window simulations with costs, turnover, and benchmarks:  
  - Equal Weight  
  - 60/40  
  - Inverse-Volatility  
  - Min-Variance  

---

## Conclusion & Next Steps
-  Summarize insights  
-  Highlight trade-offs (theory vs. implementation)  
-  Suggest future extensions (factor models, robust optimization, Black–Litterman)  


## 📊 Statistical Analysis of Portfolio and Individual Securities

### Introduction
This notebook begins the **Portfolio Optimization Project** by analyzing the statistical properties of both the **individual securities** and the **overall portfolio**. The purpose is the following:

1. **Understand the data**: Before applying optimization techniques, we need to explore the underlying behavior of the assets in terms of returns, risk, and correlations.  
2. **Establish baselines**: These descriptive statistics will serve as benchmarks against which optimization and constraints can be evaluated.

### Objectives
- Ingest and clean price data from Financial Modeling Prep
- Compute **daily log returns** and derive annualized metrics.  
- Perform statistical analysis at both the **asset level** and **portfolio level**:  
  - Expected return  
  - Volatility (standard deviation)  
  - Sharpe ratio (with constant risk-free rate)  
  - Skewness & kurtosis  
  - Correlation matrix
  - Sortino Ratio
  - Appraisal Ratio
- Visualize key results using plots and tables.  


## STEP #1: Portfolio and Individual ETFs data analysis

In [1]:
""" 
Import all necessary modules and get fmp key

"""



import pandas as pd
import numpy as np
import os
import sys
import dotenv
from dotenv import load_dotenv

import sys
# Go one level up to the project root
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from common.PortConnect import Port_Connect
from common.Portfolio import Portfolio_Stats

fmp_key = os.getenv("API_KEY")

port = Port_Connect(api_key=fmp_key)
portfolio = Portfolio_Stats()


#### Selecting appropiate benchmarks

SPY – SPDR S&P 500 ETF Trust - North American Large Cap Equities

EFA – iShares MSCI EAFE ETF - Developed Market ex-US Equities

EEM – iShares MSCI Emerging Markets ETF - Emerging Markets Equities

IEF – iShares 7-10 Year Treasury Bond ETF - US Treasuries (Fixed Income)

LQD – iShares iBoxx $ Investment Grade Corporate Bond ETF - US Investment Grade Corporate Bonds

VNQ – Vanguard Real Estate ETF - Real Estate (REITs)


In [2]:
spy = port.get_closing_prices('SPY',from_date='2005-01-01')
efa = port.get_closing_prices("EFA",from_date='2005-01-01')
eem = port.get_closing_prices("EEM",from_date='2005-01-01')
ief = port.get_closing_prices("IEF",from_date='2005-01-01')
lqd = port.get_closing_prices("LQD",from_date='2005-01-01')
vnq = port.get_closing_prices("VNQ",from_date='2005-01-01')

#### Analyzing each security independently

In [3]:
## Create a dataframe that holds all the securitis in one
benchmark = pd.concat([
    spy,efa,eem,ief,lqd,vnq
],axis=1)

benchmark



symbol,SPY,EFA,EEM,IEF,LQD,VNQ
date,,,,,,
2005-01-03,120.30,53.03,22.19,85.12,112.25,55.89
2005-01-04,118.73,52.02,21.51,84.59,111.62,55.05
2005-01-05,118.01,51.98,21.25,84.73,111.71,53.22
2005-01-06,118.61,51.98,21.23,84.81,111.79,53.63
2005-01-07,118.44,51.75,21.27,84.74,111.74,53.51
...,...,...,...,...,...,...
2025-08-28,648.92,92.02,50.10,96.23,110.20,91.74
2025-08-29,645.05,91.48,49.86,96.15,109.80,92.24
2025-09-02,640.27,90.58,49.82,95.55,109.05,90.71


In [4]:
### Calculate returns from the specified ETFS
returns = benchmark.pct_change()

summary_list = []
names = []

### Calculate KPIS for portfolio analysis
for col in returns.columns:
    summary = portfolio.summary_stats(returns[col],market_index=returns["SPY"])
    summary_list.append(summary)
    names.append(col)
    
### Create summary of analysis for each ETF
pd.concat(summary_list,axis=0,keys=names)

,,Cummulative Return,Annualized Return,Annualized Vol,Annualized Semideviation,Sharpe Ratio,Skewness,Kurtosis,Cornish-Fisher VaR (5%),Historic CVaR (5%),Max Drawdown,Drawdown Duration (Days),Up Days %,Ulcer index,Calmar Ratio,Sortino Ratio,Beta,Correlation
SPY,0,date 2005-01-03 1.000000 2005-01-04 0.98...,0.085100,0.191910,0.156661,0.341538,0.004759,18.140013,0.015776,0.029536,-0.564737,1982,0.546346,14.170394,-0.150690,0.492698,1.000000,1.000000
EFA,0,date 2005-01-03 1.000000 2005-01-04 0.98...,0.026717,0.213222,0.169907,-0.018745,-0.049664,16.915022,0.018314,0.032608,-0.631823,6392,0.525577,26.023767,-0.042285,0.219021,0.975406,0.877914
EEM,0,date 2005-01-03 1.000000 2005-01-04 0.96...,0.039995,0.277649,0.208785,0.046490,0.536282,21.215943,0.019269,0.040447,-0.672349,4829,0.522885,26.267706,-0.059486,0.275603,1.179797,0.815473
IEF,0,date 2005-01-03 1.000000 2005-01-04 0.99...,0.006002,0.068784,0.045256,-0.514769,0.099545,5.677263,0.006736,0.009627,-0.277182,1857,0.510000,10.359106,-0.021655,0.119714,-0.105227,-0.293587
LQD,0,date 2005-01-03 1.000000 2005-01-04 0.99...,-0.000783,0.086591,0.071293,-0.419181,0.048664,60.592085,0.002545,0.012052,-0.293712,2366,0.520000,10.913831,0.002666,0.032580,0.086416,0.191523
VNQ,0,date 2005-01-03 1.000000 2005-01-04 0.98...,0.024140,0.289966,0.235021,-0.024209,0.020247,20.000818,0.023409,0.043539,-0.757649,2899,0.525000,25.219963,-0.031861,0.221751,1.128119,0.746631


In [5]:
benchmark

symbol,SPY,EFA,EEM,IEF,LQD,VNQ
date,,,,,,
2005-01-03,120.30,53.03,22.19,85.12,112.25,55.89
2005-01-04,118.73,52.02,21.51,84.59,111.62,55.05
2005-01-05,118.01,51.98,21.25,84.73,111.71,53.22
2005-01-06,118.61,51.98,21.23,84.81,111.79,53.63
2005-01-07,118.44,51.75,21.27,84.74,111.74,53.51
...,...,...,...,...,...,...
2025-08-28,648.92,92.02,50.10,96.23,110.20,91.74
2025-08-29,645.05,91.48,49.86,96.15,109.80,92.24
2025-09-02,640.27,90.58,49.82,95.55,109.05,90.71


#### Cummulative Return of a 100 USD investment

In [6]:
### Initial investment

initial_investment = 100

cum_return_etfs = (1 + returns.fillna(0)).cumprod() * initial_investment
cum_return_etfs

symbol,SPY,EFA,EEM,IEF,LQD,VNQ
date,,,,,,
2005-01-03,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
2005-01-04,98.694929,98.095418,96.935557,99.377350,99.438753,98.497048
2005-01-05,98.096426,98.019989,95.763858,99.541823,99.518931,95.222759
2005-01-06,98.595179,98.019989,95.673727,99.635808,99.590200,95.956343
2005-01-07,98.453865,97.586272,95.853988,99.553571,99.545657,95.741635
...,...,...,...,...,...,...
2025-08-28,539.418121,173.524420,225.777377,113.052162,98.173719,164.143854
2025-08-29,536.201164,172.506129,224.695809,112.958177,97.817372,165.038468
2025-09-02,532.227764,170.808976,224.515548,112.253289,97.149220,162.300948


In [7]:
import plotly.express as px


In [8]:
cum_return_etfs.select_dtypes('number').columns

Index(['SPY', 'EFA', 'EEM', 'IEF', 'LQD', 'VNQ'], dtype='object', name='symbol')

In [ ]:
### Create chart showing a cummulative return of different ETFs to track
numerical_columns = cum_return_etfs.select_dtypes('number').columns

fig = px.line(
    cum_return_etfs.reset_index(),
x=cum_return_etfs.index.name,
y=numerical_columns,
title='100 USD Investment on different ETFs')
fig.show()